questions:

- Show the code, discuss results
- Discuss analytical std
    - Discuss why first try does not give the same answer. If we take a big amoung of models, we cannot say that average passage is 
- Discuss new idea for out state
- Ask if I can have one week more
- Ask when is the defense
- Ask what is the title of the project
- Requirements for the report? Taille min? Juste un truc à viser?
- Conseils pour le rapport ? Structure, schémas ?
- Est-ce que c'est bien d'avoir faire une GUI ? Est-ce que c'est pertinent d'en parler dans le rapport comment extend mon code (translocation model et aussi GUI) ?
- Je peux demander à une autre personne de faire l'animation ?
- On est d'accord que le temps c'est un "sachant que" ?


In [1]:
%matplotlib widget

from experiments import VelocityVsATPADPRatio, SC2RVsDiscSpiral, \
    VelocityVsPotential, DefectlessVsDefective, NonIdeal

# Comparison between SC/2R and Disc-Spiral models

In [2]:
sc2r_vs_disc_spiral = SC2RVsDiscSpiral()

# Relation between ATP/ADP concentrations ratio and velocity

In [3]:
velocity_vs_atp_adp_ratio = VelocityVsATPADPRatio()

# Velocity in presence of a potential

In [4]:
velocity_vs_barrier = VelocityVsPotential()

# Defective protomer

In [5]:
defectless_vs_defective = DefectlessVsDefective()

# Non-Ideal Models

In [6]:
non_ideal = NonIdeal()

# Report

# Introduction

# 2 Theroy of master equation and kinetic scheme

Consider a stochastic system evolving continuously in time, jumping from one discrete state to another. The state of the system at time $t$ is denoted by the random variable $S(t)=i$, where $i=1,2,\dots,N$, and we assume that the state entirely determines the system, i.e. the dynamics of the system is independent of its history. In other terms, the system is a continuous time Markov chain.
The transitions between the states represent physical or chemical reactions, that we assume are all reversible, and independent of each other. Each reaction $i\to j$ follows an exponential distribution of rate $k_{ij}$.
The dynamics is the following: the system starts in a given state $i$ at time $t=0$, sojourns there until one reaction $i\to j$ occurs, and then jumps to state $j$. The system then sojourns in state $j$ until another reaction $j\to k$ occurs, and so on. 

We can represent such a system with a kinetic scheme, a directed graph where each node reprsents a state, and each reaction $i\to j$ is represented by a directed edge from $i$ to $j$, weighted with the reaction rate $k_{ij}$. We denote the successors and predecessors of a state $i$ by $R_i^+$ and $R_i^-$ respectively. 
We assume that the kinetic scheme is connected, i.e. for any two states $i$ and $j$, there exists a path from $i$ to $j$. In the contrary case, the system would be composed of several independent subsystems, and we could study each subsystem separately. 

Given the stochastic nature of such a system, we are interested in the probability of being in state $j$ at a given time $t$, denoted by $p_j(t):=\mathbb{P}(S(t)=j)$ to simplify the notation. Its time evolution is given by the $N$ coupled differential master equations:
$$
    \dot{p}_j(t) = \sum_{i\in R_j^-}p_i(t)k_{ij} - p_j(t)\sum_{k\in R_j^+}k_{jk} = \sum_{i=1}^N \left(p_i(t)k_{ij} - p_j(t)k_{ji}\right)
$$
where we used the convention $k_{ii}=0$ for all $i$ and $k_{ij}=0$ if there is no edge from $i$ to $j$ in the kinetic scheme.
We can rewrite the master equation in matrix form:
$$
    \dot{\mathbf{p}}(t) = \mathbf{M}\mathbf{p}(t)
$$
where $\mathbf{p}(t)$ is the column vector of components $p_i(t)$ and $\mathbf{M}$ is the matrix of components $M_{ij}=k_{ji} - \delta_{ij}\sum_{k=1}^N k_{ik}$, where $\delta_{ij}$ is the Kronecker delta.
Knowing the initial condition $\mathbf{p}(0)$, we can solve the differential equation and obtain the solution:
$$
    \mathbf{p}(t) = e^{\mathbf{M}t}\mathbf{p}(0)
$$

These systems have interesting properties, the most important being the existence and uniqueness of an attracting steady-state distribution $\mathbf{p}^*$, which is in addition globally stable, i.e. any initial condition converges to $\mathbf{p}^*$ as $t\to\infty$. The properties of such systems are discussed in detail in \cite{Schnakenberg1976}, but for mathematical elegance, we give an alternative proof of the existence and uniqueness of the steady-state distribution in Appendix \ref{???} using solely linear algebra. 
To sum up, the stochastic matrix $M$ has rank $N-1$ and thus the steady-state distribution is given by the unique solution (up to a constant) solution of the linear system:
$$
    \mathbf{M}\mathbf{p}^* = 0
$$
and one has to take the normalized solution in order to have a probability distribution. 
A convenient way to directly find the steady-state distribution is to note that summing the first $N-1$ rows of the stochastic matrix gives back the last row with opposite sign
$$
    \sum_{i=1}^{N-1} M_{ij} = \sum_{i=1}^{N-1} k_{ji} - (1-\delta_{jN})\sum_{k=1}^N k_{jk} = -k_{jN} + \delta_{jN}\sum_{k=1}^N k_{Nk} = -M_{Nj}
$$
which means that we can remove the last row without changing the solution of the linear system. Therefore, we modify $M$ to $\tilde{M}$ by replaceing the last row with ones, and then the normalized steady-state distribution is given by the solution of the linear system:
$$
    \tilde{\mathbf{M}}\mathbf{p}^*
    =
    \begin{bmatrix}
        M_{1,1} & \cdots & M_{1,N} \\ 
        \vdots  & \ddots & \vdots  \\ 
        M_{N-1,1} & \cdots & M_{N-1,N} \\ 
        1 & \cdots & 1
    \end{bmatrix}
    \cdot
    \begin{bmatrix}
        p_1^*\\ 
        \vdots\\ 
        p_{N-1}^*\\ 
        p_N
    \end{bmatrix}
    =
    \begin{bmatrix}
        0\\ 
        \vdots\\ 
        0\\ 
        1
    \end{bmatrix}
$$
since the newly added row constraints the solution to be normalized. This gives a general method to solve any system defined by a master equation or equivalently a kinetic scheme.



# Appendix: Master equation derivation

The master equations are the differential equations that govern the evolution of the probability distribution $p_i(t)$ of being in state $i$ at time $t$. Let's derive it somewhat rigorously.

Let's $S(t)$ be the random variable representing the state of the system at time $t$. There are $N$ discrete states, that we use letters $i,j,k$ to denote. The predecessor and successor sets of state $i$ are denoted by $R_i^-$ and $R_i^+$, respectively. The transition rates from state $i$ to state $j$ are denoted by $k_{ij}$. The transition rates are non-negative, and we use the convention that $k_{ij}=0$ if $j\notin R_i^+$, $k_{ij}=0$ if $i\notin R_j^-$, and $k_{ii}=0$ for all $i=1,2,\dots,N$.

First, consider a single reaction $i\to j$. If we associate to this reaction a random variables $\tau_ij$ denoting the time it takes for the reaction to occur, since the reaction follows an exponential distribution, we have the following:
$$
    \mathbb{P}(\tau_{ij}<t) = 1-\exp(-k_{ij}t)
$$

Then, suppose we are in state $i$, and let's study the passage from this state to a chosen successor $j^*\in R_i^+$. Consider the event $\left<i\xrightarrow{\Delta t}j^*\right> :=$ "the system has jumped from $i$ to $j^*$ some time in the interval $[t,t+\Delta t]$". This event is equivalent to the event $\{\tau_{ij^*}<\Delta t \wedge \forall j\neq j^*, \tau_{ij}>\Delta t\}$. Since the reactions are independent, the joint probability of the reaction times is simply the product of the individual probabilities. Therefore, the probability of the abovementioned event is given by
$$
    \mathbb{P}(\tau_{ij^*}<\Delta t \wedge \forall j\neq j^*, \tau_{ij}>\Delta t) = \mathbb{P}(\tau_{ij^*}<\Delta t)\prod_{j\neq j^*}\mathbb{P}(\tau_{ij}>\Delta t) = (1-\exp(-k_{ij^*}\Delta t))\exp(-\sum_{j\neq j^*}k_{ij}\Delta t) = (1 - (1 - k_{ij^*}\Delta t + o(\Delta t)))(1 - \sum_{j\neq j^*}k_{ij}\Delta t + o(\Delta t))
$$
$$
    = k_{ij^*}\Delta t + o(\Delta t)
$$
where we used the 1st order Taylor expansion of the exponential function. 

Moreover, the event $\left<i\xrightarrow{\Delta t}i\right>:=$ "the system stayed in state $i$ during an interval of length $\Delta t$" has a probability 
$$
    \mathbb{P}(\forall j\in R_i^+, \tau_{ij}>\Delta t) = \prod_{j\in R_i^+}\mathbb{P}(\tau_{ij}>\Delta t) = \exp(-\sum_{j\in R_i^+}k_{ij}\Delta t) = 1 - \sum_{j\in R_i^+}k_{ij}\Delta t + o(\Delta t)
$$

Finally, we find the desired master equation. To be in state $j^*$ at time $t+\Delta t$, there are logically only two possibilities: either the system was already in state $j^*$ at time $t$ and did not jump to any other state in the interval $[t,t+\Delta t]$, or the system was in another state $i\in R_{j^*}^-$ at time $t$ and jumped to $j^*$ in the interval $[t,t+\Delta t]$. We control the size of the interval $\Delta t$ such that the system can only do at maximum one jump in the interval $[t,t+\Delta t]$. The jump probabilities at a given times are difficult to compute, but what we know is the conditional jump probability GIVEN that we start in a given state $i$ at time $t$, namely $\left<i\xrightarrow{\Delta t} j\right>$. Therefore, we can write the probability of being in state $j^*$ at time $t+\Delta t$ as
$$
    \mathbb{P}(S(t+\Delta t)=j^*) = \mathbb{P}(S(t)=j^*)\mathbb{P}(\left<j^*\xrightarrow{\Delta t}j^*\right>) + \sum_{i\in R_{j^*}^-}\mathbb{P}(S(t)=i)\mathbb{P}(\left<i\xrightarrow{\Delta t}j^*\right>)
$$
$$
    = \mathbb{P}(S(t)=j^*)(1 - \sum_{k\in R_{j^*}^+}k_{j^*k}\Delta t) + \sum_{i\in R_{j^*}^-}\mathbb{P}(S(t)=i)(k_{ij^*}\Delta t) + o(\Delta t)
$$

Reordering the terms, we find
$$
    \frac{\mathbb{P}(S(t+\Delta t)=j^*) - \mathbb{P}(S(t)=j^*)}{\Delta t} = \sum_{i\in R_{j^*}^-}\mathbb{P}(S(t)=i)k_{ij^*} - \mathbb{P}(S(t)=j^*)\sum_{k\in R_{j^*}^+}k_{j^*k} + \frac{o(\Delta t)}{\Delta t}
$$

Taking the limit $\Delta t\to 0$, we find the master equation
$$
    \frac{d\mathbb{P}(S(t)=j^*)}{dt} = \sum_{i\in R_{j^*}^-}\mathbb{P}(S(t)=i)k_{ij^*} - \mathbb{P}(S(t)=j^*)\sum_{k\in R_{j^*}^+}k_{j^*k}
$$

Applying the same reasoning to all states $j^*=1,2,\dots,N$, changing the dummy indices, using the convention that $k_{ij}=0$ when states $i$ and $j$ are not connected, and using the notation $p_i(t) = \mathbb{P}(S(t)=i)$, we find the $N$ coupled differential equations
$$
    \dot{p}_i(t) = \sum_{j\in R_i^-}p_j(t)k_{ji} - p_i(t)\sum_{j\in R_i^+}k_{ij} = \sum_{j=1}^N \left(p_j(t)k_{ji} - p_i(t)k_{ij}\right)
$$


# Appendix: Steady-state distribution

Consider a system represented by its $N$-dimensional master equation $\dot{\mathbf{p}}(t) = \mathbf{M}\mathbf{p}(t)$, where $\mathbf{p}(t)$ is the column vector of components $p_i(t)$ and $\mathbf{M}$ is the matrix of components $M_{ij}=k_{ji} - \delta_{ij}\sum_{k=1}^N k_{ik}$, where $\delta_{ij}$ is the Kronecker delta. We assume that the corresponding kinetic scheme of the system is connected, and we use the convention that $k_{ii}=0$.
We show that the system has a unique steady-state distribution $\mathbf{p}^*$, i.e. a solution of the equation $\mathbf{M}\mathbf{p}^* = 0$.

Proposition 1: A solution of the continuous time system $\dot{\mathbf{p}}(t) = \mathbf{M}\mathbf{p}(t)$ is steady-state if and only if it is a fixed-point of a discrete time system $\mathbf{p}^{(t+1)} = \mathbf{W}\mathbf{p}^{(t)}$ with stochastic matrix $\mathbf{W} := \mathbf{I} + \mathbf{M}\Delta t$, where $\mathbf{I}$ is the identity matrix and $\Delta t := \frac{1}{N\max_{i,j}k_{ij}}$.

Proof: 
$$
    \underbrace{\mathbf{p} \overset{!}{=} \mathbf{W}\mathbf{p}}_\text{fixed-point} = \mathbf{M}\mathbf{p} + \mathbf{p} \iff \underbrace{\mathbf{M}\mathbf{p} = 0}_\text{steady-state}
$$

Lemma 1: Any element of the stochastic matrix $\mathbf{W}$ is non-negative and the sum of any column is 1.

Proof: 
$$
    W_{ij} 
    = \delta_{ij} + M_{ij}\Delta t 
    = \delta_{ij} + (k_{ji} - \delta_{ij}\sum_{k=1}^N k_{ik})\Delta t 
    = k_{ji}\Delta t + \delta_{ij} \left(N\max_{i,j}k_{ij} - \sum_{k=1}^N k_{ik}\right)\Delta t
    \geq 0
$$
and
$$
    \sum_{i=1}^N W_{ij}
    = 1 + \sum_{i=1}^N \left(k_{ji} - \delta_{ij}\sum_{k=1}^N k_{ik}\right) \Delta t
    = 1 + \left(\sum_{i=1}^N k_{ji} - \sum_{k=1}^N k_{jk}\right) \Delta t
    = 1
$$


For each column of the stochastic matrix $\mathbf{W}$, any partial sum of its components is non-negative and summing the whole column gives 1.
Let $S$ be a subset of $\{1,2,\dots,N\}$. Then, the sum of the components of the $j$-th column of $\mathbf{W}$ is given by
$$
    \sum_{i\in S}W_{ij} 
    = \sum_{i\in S}\left(k_{ji} - \delta_{ij}\sum_{k=1}^N k_{ik}\right) \Delta t + \chi_{j\in S} 
    = \left(\sum_{i\in S}k_{ji} - \chi_{j\in S}\sum_{k=1}^N k_{jk}\right)\Delta t + \chi_{j\in S}
    = 
    \left\{
        \begin{array}{ll}
            \sum_{i\in S}k_{ji}\Delta t \geq 0 & \text{if } j\notin S \\
            1 - \sum_{i\notin S}k_{ji}\Delta t \geq 0 & \text{if } j\in S
        \end{array}
    \right.
$$
and then it immediately follows that 
$$
    S=\{1,2,\dots,N\} \implies \sum_{i\in S} W_{ij} = 1
$$

Proposition 2: There exists a fixed-point $\mathbf{p}^*$ of the discrete time system on the probability simplex $\Delta_N := \{\mathbf{p}\in\mathbb{R}^N \mid \sum_{i=1}^N p_i = 1 \wedge \forall i, p_i\geq 0\}$.

Proof: First, $\mathbf{W}$ is an endomorphism $\mathbf{W}: \Delta_N \to \Delta_N$, since for $\mathbf{p}^{(t)}\in \Delta_N$, we have
$$
    p_i^{(t+1)} 
    = \left(\sum_{j=1}^N W_{ij}p_j^{(t)}\right) 
    = \sum_{j=1}^N \underbrace{\left(W_{ij}\right)}_{\geq 0}  p_j^{(t)} 
    \geq 0
$$
and
$$
    \sum_{i=1}^N p_i^{(t+1)} 
    = \sum_{i=1}^N \left(\sum_{j=1}^N W_{ij}p_j^{(t)}\right) 
    = \sum_{j=1}^N \underbrace{\left(\sum_{i=1}^N W_{ij}\right)}_{=1}  p_j^{(t)} 
    = \sum_{j=1}^N p_j^{(t)} 
    = 1
$$

Moreover, $\mathbf{W}$ is linear thus continuous and $\Delta_N$ is a compact convex set. Therefore, the Brouwer fixed-point theorem implies the existence of a fixed-point, that we call $\mathbf{p}^*$.

Proposition 3: The coordinates of the fixed point $\mathbf{p}^*\in \Delta_N$ are strictly within $]0, 1[$.

Proof: If the kinetic scheme of the continuous-time system is connected, then so is the kinetic scheme of the discrete-time system. Therefore, by definition of connectivity, $(W^N)_{i,j}>0$, $\forall i,j$. Moreover, since $\mathbf{p}^*$ is a fixed-point, we have $\mathbf{p}^* = \mathbf{W^N}\mathbf{p}^*$. Let's assume that there exists $i$ such that $p_i^*=0$. Then
$$
    p_i^* = \sum_{j=1}^N (W^N)_{i,j}p_j^* \overset{↯}{\neq} 0 
$$
since it is a sum of non-negative terms, at least one of which is strictly positive. On the other hand, if one coordinate is 1, then all the others must be zero since the sum of the coordinates is 1, but we have just shown a contradiction when a coordinate is null. Therefore, all the coordinates of $\mathbf{p}^*$ are strictly within $]0, 1[$.

Corollary 1: The fixed-point $\mathbf{p}^*\in \Delta_N$ is unique on $\Delta_N$.

Proof: Proposition 3 restricts the existence of a fixed-point on the boundary of the probability simplex $\Delta_N$. Now assume there exists another fixed-point $\mathbf{q}^*\in \Delta_N$, different from $\mathbf{p}^*$. By linearity, any linear combination of $\mathbf{p}^*$ and $\mathbf{q}^*$ is also a fixed-point of $\mathbf{W}$. Therefore, there must exists a linear combination $\mathbf{r}^* = \alpha(\mathbf{p}^* + \beta\mathbf{q}^*)$ that lies on an edge of the probability simplex ($\mathbf{p}^*$ and $\mathbf{q}^*$ differ on at least one coordinate, $p^*_i\neq q^*_i \neq 0$, choose $\beta = -\frac{p^*_i}{q^*_i}$ and choose $\alpha$ so that $\mathbf{r}^*$ is normalized), which contradicts Proposition 3. 


Proposition 4: The fixed-point $\mathbf{p}^*\in \Delta_N$ is unique (up to a constant) on $\mathbb{R}^N$.

Proof: The proof is similar than the one of Corollay 1. Assume there exists a fixed-point $\mathbf{q}^*$ in a different orthant than the probability simplex. Then, construct the linear combination $\mathbf{r}^* = \alpha(\mathbf{p}^* + \beta\mathbf{q}^*)$ that lies on the boundary of the probability simplex ($\mathbf{p}^*$ and $\mathbf{q}^*$ differ on at least one nonzero coordinate, $p^*_i\neq q^*_i \neq 0$, choose $\beta = -\frac{p^*_i}{q^*_i}$ and choose $\alpha$ so that $\mathbf{r}^*$ is normalized), which contradicts Proposition 3.

Theorem 1: The fixed-point $\mathbf{p}^*\in \Delta_N$ is the unique steady-state distribution of the continuous-time system $\dot{\mathbf{p}}(t) = \mathbf{M}\mathbf{p}(t)$.

Proof: Prop. 4 says that the subspace of fixed-points of $\mathbf{W}$ has dimension 1. By the relation between $\mathbf{W}$ and $\mathbf{M}$ immediately implies that the kernel of the latter is also of dimension 1. Therefore the linear system $\mathbf{M}\mathbf{p}^* = 0$ has a unique solution (up to a constant), which is the normalized fixed-point $\mathbf{p}^*$.

To show the global convergence to the steady-state distribution, one has to use the Liapunov function presented in \cite{Schnakenberg1976}.

